In [15]:
# output mwlBenchmark.txt (mwBen) (no duplicate lines)

# format: derivatives of hw (inflections, or subhws);hw
# sort by 'derivatives;hw' and make every line unique
# 2015-11-28, 2015-11-30, 2015-12-01

# blazes;blaze
# blazing;blaze
# blazingly;blazing

# I can know blaze, blazing are hws, the two lines below are not allowd because hws have derivates:
# blaze;blaze
# blazing;blazing
# the two lines above are redundant.

# manually delete: asas

import pymongo
from pymongo import MongoClient

# codecs if for read and write files with unicode, encoding 'utf-8'
import codecs

#connect to MongoDB
client = MongoClient('localhost', 27017)
db = client.dpdb  # databas: dpdb
geweiDict = db.geweiDict201511 # collection (table): geweiDict201511
#geweiDict = db.geweiDictTest

fdict = codecs.open('E:\\1Now\\taglist\\mwlBenchmarkraw.txt', 'w', 'utf-8')
    
results = geweiDict.find({}, {"_id":0, "hw":1, "infl":1, "subhw":1}) #.limit(1000)
# sort method will use much more memory
#results = geweiDict.find({}, {"_id":0, "hw":1, "prn":1, "alles":1, "idpvs":1, "sublist":1}).sort("hw", 1).limit(1000)


hwcnt = 0 # number of head words (hws)
# hwcapcnt = 0 # number of hws capitalized
inflcnt = 0 # number of inflections
subhwcnt = 0 # number of subhws
subhw1cnt = 0 # number of subhws whose first letter is not same as hw
onlyhwcnt = 0

# subhw1hw = []

for result in results:
    inflflag = 0
    subhwflag = 0
    
    word = result.get("hw")    
    if word:
        #print word
#         fdict.write(word + ';' + word + '\n')
        hwcnt += 1      
    else:
        print "no head word"
        continue

    # get items of result
    infls = result.get("infl")
    subhws = result.get("subhw")     
    
    if infls:
        for infl in infls:
            if infl == word:
#                 print 'infl == word', infl
                continue
            fdict.write(infl + ';' + word + '\n')
            inflcnt += 1   
            inflflag = 1
            
    if subhws:
        for subhw in subhws:
            #if subhw[0] != word[0]: continue       
            if subhw == word:
#                 print 'subhw == word', infl
                continue           
            if subhw[0] != word[0]:
#                 print 'subhw is not like word: ', subhw, '...', word
                subhw1cnt += 1
#                 subhw1hw.append(subhw+';'+word)
            fdict.write(subhw + ';' + word + '\n')
            subhwcnt += 1      
            subhwflag = 1
            
    # hw;hw (only when hw has no inflections and no subhws)
    if inflflag == 0 and subhwflag == 0:
        fdict.write(word + ';' + word + '\n')
        onlyhwcnt += 1
    
fdict.close()

print '--- mwBen statistics ---'
print "total head words: ", hwcnt
# print 'total hws capitalized: ', hwcapcnt
print "total inflections: ", inflcnt
print 'total subhws: ', subhwcnt
print 'total subhws whose 1st letters are different from hws: ', subhw1cnt
print 'hw without any derivatives: ', onlyhwcnt


with codecs.open('E:\\1Now\\taglist\\mwlBenchmarkraw.txt', 'r', 'utf-8') as f:
    mwls = f.read().splitlines()
print '\n\nnumber of lines in mwBen before adding 84en_fr.txt: ', len(mwls)
mwls = sorted(set(mwls))
print 'number of unique lines in mwBen before adding 84en_fr.txt: ', len(mwls)

# read en_fr.txt
with codecs.open('E:\\1Now\\taglist\\84en_fr.txt', 'r', 'utf-8') as f:
    enfrs = f.read().splitlines()
print 'number of lines in 84en_fr.txt: ', len(enfrs)
enfrs = sorted(set(enfrs))
print 'number of unique lines in 84en_fr.txt: ', len(set(enfrs))

compareWordlist(l1=enfrs, l2=mwls)

# sort and make unique
lines = sorted(set(mwls + enfrs))
print 'number of lines in mwBen after adding 84en_fr.txt: ', len(lines)

fd = codecs.open('E:\\1Now\\taglist\\mwlBenchmark.txt', 'w', 'utf-8')
for line in lines:
    fd.write(line + '\n')
fd.close()


# subhw1hw = sorted(set(subhw1hw))
# fd = codecs.open('E:\\1Now\\taglist\\subhw1hw.txt', 'w', 'utf-8')
# for x in subhw1hw:
#     fd.write(x + '\n')
# fd.close()


--- mwBen statistics ---
total head words:  39195
total inflections:  64355
total subhws:  9544
total subhws whose 1st letters are different from hws:  334
hw without any derivatives:  11182


number of lines in mwBen before adding 84en_fr.txt:  85081
number of unique lines in mwBen before adding 84en_fr.txt:  53972
number of lines in 84en_fr.txt:  84
number of unique lines in 84en_fr.txt:  83
---common words of list-1 and list-2---: 15
---words only in list-1---: 68
---words only in list-2---: 53957
number of lines in mwBen after adding 84en_fr.txt:  54040


In [15]:
# explore mwBen

import codecs
import string

with codecs.open('E:\\1Now\\taglist\\mwlBenchmark.txt', 'r', 'utf-8') as f:
    lines = f.read().splitlines()

deriv = [] # derivative
benhw = [] # hw
deriv1hw = [] # derivative (inflection, subhw) first letter different from its hw
hwcap = [] # hw whose first letter is capitalized
for line in lines:
    x1 = line.split(';')[0]
    x2 = line.split(';')[1]
    deriv.append(x1)
    benhw.append(x2)
    if x1[0] != x2[0]: deriv1hw.append(line)
    if x2[0] in string.uppercase[0:26]: hwcap.append(x2)
derivS = sorted(deriv)
benhwSU = sorted(set(benhw)) # Sorted, Unique
hwcap = sorted(set(hwcap))

print '--- explore mwBen ---'
print 'number of derivative column: ', len(deriv)
print 'number of hw column: ', len(benhw)
print 'unique number of hw column: ', len(benhwSU)
print 'number of derivatives (inflections and subhws) whose 1st letter different from hws: ', len(deriv1hw)
print 'number of capitalized hws: ', len(hwcap)

fout = codecs.open('E:\\1Now\\taglist\\mwlStat.txt', 'w', 'utf-8')
fout.write('--- statistics ---\n')
fout.write('number of lines in mwBen: ' + str(len(deriv)) + '\n')
fout.write('unique number of hw: ' + str(len(benhwSU)) + '\n')
fout.write('number of derivatives (inflections and subhws) whose 1st letter different from hws: ' + str(len(deriv1hw)) + '\n')
fout.write('number of capitalized hws: ' + str(len(hwcap)) + '\n')

fout.write('Contents\n--------\n')
fout.write("Q: Are there duplicate derivatives who aren't hws in mwBen?\n")
fout.write("Q: List derivatives (inflections and subhws) whose first letter is different from the first letter of their hws\n")
fout.write("Capitalized hws\n\n")

# Q: Are there duplicate derivatives who aren't hws in mwBen?
dupderiv = [x for x in derivS if x not in benhwSU and derivS.count(x)>1]
fout.write("--- Duplicate derivatives not being hws: ---\n")
for x in dupderiv:
    fout.write(x + '\n')

# Q: List subhws whose first letter is different from the first letter of their hws
fout.write("\n--- derivatives (inflections and subhws) whose 1st letter different from hws: ---\n")
fout.write("derivatives (inflections and subhws);hw\n\n")
for x in subhw1hw:
    fout.write(x + '\n')

fout.write("\n--- Capitalized hws: ---\n")
for x in hwcap:
    fout.write(x + '\n')
           
fout.close()    

--- explore mwBen ---
number of derivative column:  53972
number of hw column:  53972
unique number of hw column:  34398
number of derivatives (inflections and subhws) whose 1st letter different from hws:  337
number of capitalized hws:  1487


In [1]:
# check word lists 
# these word lists include (number in parentheses are the number of files)
#   * words of 18 topics from mwl website (18)
#   * 3140 core words from mwl website (1)
#   * words of "The Spelling of Different Sounds in English" (1)
#   * words of word roots, prefixes, suffixes (3)
#   sub-total: 23
#
#   * words of mwl single images (1)
#   * words of mwl group images (108)
#   sub-total: 109
#
#   * homographs (1)
#   * 3787 core words I collect (1)
#   sub-total: 2
# 
#   Total: 135

import codecs
import re
import string


def checkBymwBen(path, wordlist):
    '''get sorted unique head words of mwl for a word list which may include inflections and/or sub hws'''
    
    hwreturn = [] # hws returned by this function    
    
    # word list I want to check
    with codecs.open(path + wordlist, 'r', 'utf-8') as f:
        txtraw = f.read().splitlines()
    print '\n\n------', wordlist, '------'
    print '* has words : ', len(txtraw)
    print '* has unique word number: ', len(set(txtraw))
    dupinraw = set([x for x in txtraw if txtraw.count(x) > 1])
    if dupinraw: print '* has duplicates: ', dupinraw


    # with codecs.open("E:\\1Now\\taglist\\mwl77Entry.txt", 'r', 'utf-8') as f:
    with codecs.open("E:\\1Now\\taglist\\mwlBenchmark.txt", 'r', 'utf-8') as f:    
        lines = f.read().splitlines()

    # create mwBen dictionary
    mwBens = []
    hws = [] # sorted unique hws
    hwsC = [] # sorted unique capitalized hws
    for line in lines:
        entry = {}
        hw = line.split(';')[1]
        entry['hw'] = hw
        entry['deriv'] = line.split(';')[0] # derivative, or hw that has no derivatives
        mwBens.append(entry)
        hws.append(hw)
        if hw[0] in string.lowercase[0:26]:
            hwsC.append(hw.capitalize())
        
    hws = sorted(set(hws))
    hwsC = sorted(set(hwsC))        

#     print 'mwBens: ', len(mwBens)
#     print 'hws: ', len(hws)
#     print 'hwsC: ', len(hwsC)


    # begin to check the word list against mw benchmark dictionary
    #  Word; Its-hw (Head Word)
    #
    # Three cases (the first two cases return hw)
    # 1. word is exactly same as hw or is a capitalized hw
    # 2. word is a derivative or is a capitalized derivative
    # 3. word is not in mwBen


    wordhw = [] # word is exactly same as hw
    wordhwC = [] # word is same as capitalized hw
    wordderiv = [] # derivatives or capitalized derivatives
    wordnotBen = [] # word is not in mwBen

    for word in txtraw:
        hwflag = 0
        derivflag = 0
        
        if word in hws:
            wordhw.append(word)
            hwreturn.append(word)
            hwflag = 1
        elif word.capitalize() in hws: # like leo
            wordhw.append(word)
            hwreturn.append(word.capitalize())
            hwflag = 1            
        elif word in hwsC: # like Walk
            wordhwC.append(word)
            hwreturn.append(word.lower())
            hwflag = 1

        if hwflag == 0:
            dercnt = 0
            for mwben in mwBens:                
                if word == mwben['deriv'] or word == mwben['deriv'].capitalize() or word.capitalize() in mwben['deriv']:
                    wordderiv.append(word)
                    hwreturn.append(mwben['hw'])
                    derivflag = 1
                    print 'derivative: ', word, '--- hw: ', mwben['hw']
                    dercnt += 1
            if dercnt > 1: print word, '--- derivate has multiple hws --- ', dercnt

        if hwflag == 0 and derivflag == 0:
            if word.find(' ') == -1: wordnotBen.append(word)
            else:
                print '>>> >>> >>> >>> >> not a single word: ', word
                for x in word.split(' '):
                    if x in hws: hwreturn.append(x)
                    elif x.capitalize() in hws: hwreturn.append(x.capitalize())
                    elif x in hwsC: hwreturn.append(x.lower())
                    else:
                        wdflagx = 0 
                        for mwben in mwBens:                
                            if x == mwben['deriv'] or x == mwben['deriv'].capitalize() \
                            or x.capitalize() in mwben['deriv']:
                                hwreturn.append(mwben['hw'])
                                wdflagx = 1
                        if wdflagx == 0: print '**************^^^^^^^!!@@@@@*************** really is not in mwBen: ', x
                                
                            
            
    
    if wordhw: print '* same as hw: ', len(wordhw)  #,' -- ', wordhw
    if wordhwC: print '* same as capitalized hw: ', len(wordhwC)  #,' -- ', wordhwC
    if wordderiv: print '* words are derivatives: ', len(wordderiv) #,' -- ', wordderiv
        
    if len(txtraw) == len(wordhw) + len(wordhwC) + len(wordderiv) + len(wordnotBen): 
        if wordnotBen:
            print '**************##########***************** not in mwBen: ', len(wordnotBen), ' -- ', wordnotBen
#         print '--- len(txtraw) == len(wordhw) + len(wordhwC) + len(wordderiv) + len(wordnotBen) ---\n'
    else: print "--- words' numbers don't mattch, check ---\n"
    
    dupl = [x for x in hwreturn if hwreturn.count(x)>1]
    if dupl:
        print 'how many duplicates in raw hwreturn --- ', len(dupl), ' - ', len(set(dupl)), ' = ', len(dupl)-len(set(dupl))
        print dupl

    print '\nhow many hws supposed to be returned--- ', len(txtraw) - len(wordnotBen) - (len(dupl)-len(set(dupl)))
    hwreturn = sorted(set(hwreturn))
    print 'acutal return: ', len(hwreturn)
    return hwreturn
    
print checkBymwBen('E:\\1Now\\taglist\\', 'testmwBen.txt')

# result = checkBymwBen('E:\\1Now\\taglist\\', '3744BasicWords.txt')
# f = codecs.open("E:\\1Now\\taglist\\2939BasicWords.txt", 'w', 'utf-8')
# for x in result:
#     f.write(x + '\n')
# f.close()


# import os
# from os import listdir
# from os.path import isfile, join

# wlpath = "E:\\1Now\\taglist\\mwlimgGrpwlRaw\\"
# for wlname in os.listdir(wlpath):
#     if isfile(join(wlpath, wlname)):
#         result = checkBymwBen(wlpath, wlname)
#     else: continue
    
#     f = codecs.open("E:\\1Now\\taglist\\mwlimgGrpwl\\" + wlname, 'w', 'utf-8')
#     for x in result:
#         f.write(x + '\n')
#     f.close()




------ testmwBen.txt ------
* has words :  20
* has unique word number:  19
* has duplicates:  set([u'taking'])
derivative:  Taking --- hw:  take
derivative:  taking --- hw:  take
derivative:  tAke --- hw:  shot
derivative:  taking --- hw:  take
derivative:  20/20 --- hw:  twenty-twenty
derivative:  Flurries --- hw:  flurry
derivative:  axes --- hw:  ax
derivative:  axes --- hw:  axis
axes --- derivate has multiple hws ---  2
* same as hw:  10
* same as capitalized hw:  3
* words are derivatives:  8
--- words' numbers don't mattch, check ---

how many duplicates in raw hwreturn ---  12  -  5  =  7
[u'take', u'take', u'takings', u'takings', u'take', u'blazing', u'blazing', u'take', u'twenty-twenty', u'twenty-twenty', u'impressionism', u'impressionism']

how many hws supposed to be returned---  13
acutal return:  14
[u'against', u'ax', u'axis', u'blaze', u'blazing', u'flurry', u'impressionism', u'revolt', u'shot', u'take', u'taken', u'takings', u'took', u'twenty-twenty']


In [6]:
a = {'k1': 'v1', 'k1':'v2'}
print a['k1']

v2


In [3]:
s = "  \t a string example\t  "
print s
s = s.strip()
print s

  	 a string example	  
a string example


In [11]:
# check a word list:
# * transfer to mwl hws
# 


import codecs
import re
import string

def tuneBymwBen(wlist=[], path='', wfile=''):
    ''''''
    rnt = []
    hwreturn = [] # hws returned by this function    
    
    if wfile:
        print '\n\n------', path+wfile, '------'
        with codecs.open(path+wfile, 'r', 'utf-8') as f:
            txtraw = f.read().splitlines()
        txtraw = [x.strip() for x in txtraw]
    elif wlist:
        txtraw = [x.strip() for x in wlist]
    else:
        print 'function argumenents error'
        return -1
    
    print '* has words : ', len(txtraw)
    print '* has unique word number: ', len(set(txtraw))
    dupinraw = set([x for x in txtraw if txtraw.count(x) > 1])
    if dupinraw: print '* has duplicates: ', dupinraw
   
        
    with codecs.open("E:\\1Now\\taglist\\mwlBenchmark.txt", 'r', 'utf-8') as f:    
        lines = f.read().splitlines()

    # create mwBen dictionary
    mwBens = []
    hws = [] # sorted unique hws
    hwsC = [] # sorted unique capitalized hws
    for line in lines:
        if len(line) < 1:
            print 'len(line)<1', line 
            return -1
        entry = {}
        hw = line.split(';')[1]
        entry['hw'] = hw
        entry['deriv'] = line.split(';')[0] # derivative, or hw that has no derivatives
        mwBens.append(entry)
        hws.append(hw)
        if hw[0] in string.lowercase[0:26]:
            hwsC.append(hw.capitalize())
        
    hws = sorted(set(hws))
    hwsC = sorted(set(hwsC))        


    # begin to check the word list against mw benchmark dictionary
    #  Word; Its-hw (Head Word)
    #
    # Three cases
    # 1. word is exactly same as hw or is a capitalized hw (like 'Walk') or word.capitalize() is in hw (like 'leo')
    # 2. word is a derivative or is a capitalized derivative (like 'Walks') or word.capitalize() is in derivate (like 'leos')
    # 3. word is not in mwBen


    wordhw = [] # word is exactly same as hw
    wordhwL = [] # word.lower() is a hw, like 'Walk'
    wordhwC = [] # word.capitalize() is a hw, like 'leo'
    
    wordDeriv = [] # raw derivatives or capitalized derivatives or derivative.lower()
    fineDeriv = [] # fine derivateivs are in mwBen
    hwofDeriv = [] # hws of derivatives
    wordnotBen = [] # word is not in mwBen

    for word in txtraw:
        hwflag = 0 # suppose that word is not a hw
        derivflag = 0 # suppose that word is not a derivative
        
        # first check if word is a hw
        if word in hws:
            wordhw.append(word)
            hwreturn.append(word)
            hwflag = 1
        elif word.capitalize() in hws: # like leo
            wordhwC.append(word)
            hwreturn.append(word.capitalize())
            hwflag = 1            
        elif word.capitalize() in hwsC: # like Walk, tAke
            wordhwL.append(word)
            hwreturn.append(word.lower())
            hwflag = 1

        # if word isn't a hw, then check if it's a derivative
        if hwflag == 0:
            dercnt = 0
            for mwben in mwBens:                
                if word == mwben['deriv'] or word == mwben['deriv'].capitalize() or word.capitalize() == mwben['deriv']:
                    wordDeriv.append(word)
                    fineDeriv.append(mwben['deriv'])
                    hwofDeriv.append(mwben['hw'])
                    derivflag = 1
                    print 'derivative: ', word, '--- hw: ', mwben['hw']
                    dercnt += 1
            if dercnt > 1: print word, '--- derivate has multiple hws --- ', dercnt

        if hwflag == 0 and derivflag == 0:
            if word.find(' ') == -1:
                wordnotBen.append(word)
                print '**************^^^^^^^!!@@@@@*************** is not in mwBen: ', word
            else:
                print '>>> >>> >>> >>> >> not a single word: ', word
                for x in word.split(' '):
                    if x in hws: hwreturn.append(x)
                    elif x.capitalize() in hws: hwreturn.append(x.capitalize())
                    elif x in hwsC: hwreturn.append(x.lower())
                    else:
                        wdflagx = 0 
                        for mwben in mwBens:                
                            if x == mwben['deriv'] or x == mwben['deriv'].capitalize() \
                            or x.capitalize() == mwben['deriv']:
                                wordDeriv.append(word)
                                fineDeriv.append(mwben['deriv'])
                                hwofDeriv.append(mwben['hw'])
                                wdflagx = 1
                        if wdflagx == 0:
                            wordnotBen.append(x)
                            print '**************^^^^^^^!!@@@@@*************** really is not in mwBen: ', x
                                
                            
            
    
    if wordhw: print '* same as hw: ', len(wordhw)  #,' -- ', wordhw
    if wordhwL: print '* capitalized hw: ', len(wordhwL)  #,' -- ', wordhwL
    if wordhwC: print '* hw typo, should be capitalized: ', len(wordhwC)  #,' -- ', wordhwC
    
    if wordDeriv: print '* words are derivatives: ', len(wordDeriv) #,' -- ', wordderiv
        
    if len(txtraw) == len(wordhw) + len(wordhwL) + len(wordhwC) + len(wordDeriv) + len(wordnotBen): 
        if wordnotBen:
            print '**************##########***************** not in mwBen: ', len(wordnotBen), ' -- ', wordnotBen
    else: print "--- words' numbers don't mattch, check ---\n"

        
    rntdic = {}
    rntdic['fineallhw'] = sorted(set(hwreturn + hwofDeriv))   
    rntdic['finederiv'] = sorted(set(fineDeriv))
    rntdic['rawhwderiv'] = sorted(set(wordhw + wordhwL + wordhwC + wordDeriv))
    rntdic['notinben'] = sorted(set(wordnotBen))
   
    
    hwreturn += hwofDeriv    
    dupl = [x for x in hwreturn if hwreturn.count(x)>1]
    if dupl:
        print 'how many duplicates in raw hwreturn --- ', len(dupl), ' - ', len(set(dupl)), ' = ', len(dupl)-len(set(dupl))
        print dupl

    print '\nhow many hws supposed to be returned--- ', len(txtraw) - len(wordnotBen) - (len(dupl)-len(set(dupl)))
    hwreturn = sorted(set(hwreturn))
    print 'acutal return: ', len(hwreturn)
    
    return rntdic
    


def tuneReport(wlist=[], path='', wfile=''):
    if wlist: fname = 'tuneReport-list.txt'
    elif wfile: fname = 'tuneReport-' + wfile
    else:
        print 'arguments error'
        return -1
    rnt = tuneBymwBen(wlist, path, wfile)

    fineallhw = rnt['fineallhw'] 
    finederiv = rnt['finederiv'] 
    rawhwderiv = rnt['rawhwderiv']
    notinben = rnt['notinben']

    
    fd = codecs.open('E:\\1Now\\taglist\\report\\' + fname, 'w', 'utf-8')
    if notinben:
        fd.write('---not in mwBen---:' + str(len(notinben)) + '\n')
        for x in notinben: fd.write(x + '\n')
    if fineallhw:
        fd.write('\n\n---fine hws in mwBen---:' + str(len(fineallhw)) + '\n')
        for x in fineallhw: fd.write(x + '\n')    
    if finederiv:
        fd.write('\n\n---fine derivatives in mwBen---:' + str(len(finederiv)) + '\n')
        for x in finederiv: fd.write(x + '\n')    

    fd.close()
    
    
# print tuneBymwBen(path='E:\\1Now\\taglist\\', wfile='testmwBen.txt')
    
# tuneReport(path='E:\\1Now\\taglist\\', wfile='570awl.txt')
# tuneReport(path='E:\\1Now\\taglist\\', wfile='TGraw.txt')
tuneReport(wlist=txt2wl('E:\\1Now\\mae\\article\\', 'pd12.txt'))
    
# result = checkBymwBen('E:\\1Now\\taglist\\', '3744BasicWords.txt')
# f = codecs.open("E:\\1Now\\taglist\\2939BasicWords.txt", 'w', 'utf-8')
# for x in result:
#     f.write(x + '\n')
# f.close()


# import os
# from os import listdir
# from os.path import isfile, join

# wlpath = "E:\\1Now\\taglist\\mwlimgGrpwlRaw\\"
# for wlname in os.listdir(wlpath):
#     if isfile(join(wlpath, wlname)):
#         result = checkBymwBen(wlpath, wlname)
#     else: continue
    
#     f = codecs.open("E:\\1Now\\taglist\\mwlimgGrpwl\\" + wlname, 'w', 'utf-8')
#     for x in result:
#         f.write(x + '\n')
#     f.close()


* has words :  132
* has unique word number:  132
**************^^^^^^^!!@@@@@*************** is not in mwBen:  
**************^^^^^^^!!@@@@@*************** is not in mwBen:  -
**************^^^^^^^!!@@@@@*************** is not in mwBen:  09
**************^^^^^^^!!@@@@@*************** is not in mwBen:  2
**************^^^^^^^!!@@@@@*************** is not in mwBen:  29
**************^^^^^^^!!@@@@@*************** is not in mwBen:  3
**************^^^^^^^!!@@@@@*************** is not in mwBen:  47-3
**************^^^^^^^!!@@@@@*************** is not in mwBen:  53-4
**************^^^^^^^!!@@@@@*************** is not in mwBen:  DOCTER
**************^^^^^^^!!@@@@@*************** is not in mwBen:  PD1
**************^^^^^^^!!@@@@@*************** is not in mwBen:  PD2
derivative:  caricatured --- hw:  caricature
derivative:  characters --- hw:  character
derivative:  compared --- hw:  compare
**************^^^^^^^!!@@@@@*************** is not in mwBen:  dendrites
derivative:  dolls --- hw:  dol

In [10]:
import re

def txt2wl(path, txtfile):
    '''extract a word list from a text, return a sorted unqiue raw word list'''
    with codecs.open(path + txtfile, 'r', 'utf-8') as f:
        parags = f.read().splitlines() # each paragraph as one line
    
    wlraw = [] # raw word list from the text
    for parag in parags:
        regxp = re.compile(r'[^\w\'\-]+', re.U) # regular expression pattern: ^\w\'\- means not words ' -
        parag = regxp.sub(' ', parag) # replace all not words ' - with a space
        wlraw += parag.split(' ')
    
    rnt = sorted(set(wlraw))
    return rnt

# print txt2wl('E:\\1Now\\mae\\article\\', 'pd12.txt')


def beaExclu(wlist=[], wfile=''):
    '''This function only excludes hws and their derivatives exactly in mwl.
    So it's better to use tuneBymwBen() first to get a fine word list'''
    
    if wfile:
        with codecs.open(path + wfile, 'r', 'utf-8') as f:
            words = f.read().splitlines()
        words = [x.strip() for x in words]
    elif wlist:
        words = [x.strip() for x in wlist]
        
    with codecs.open("E:\\1Now\\mae\\beautifulExclu.txt", 'r', 'utf-8') as f:
        bexclus = f.read().splitlines()
    
    word4learn = []
    for word in words:
        if word not in bexclus: word4learn.append(word)
    
    return sorted(set(word4learn))
    
    
# print beaExclu(wlist = txt2wl('E:\\1Now\\mae\\article\\', 'pd12.txt'))
rnt = tuneBymwBen(wlist=txt2wl('E:\\1Now\\mae\\article\\', 'pd12.txt'))
# rnt = tuneBymwBen(path='E:\\1Now\\taglist\\', wfile='TGraw.txt')


print '\n--------- before excluded -------\n'
print rnt['fineallhw']

print '\n--------- after excluded -------\n'
print beaExclu(rnt['fineallhw'])



* has words :  132
* has unique word number:  132
**************^^^^^^^!!@@@@@*************** is not in mwBen:  
**************^^^^^^^!!@@@@@*************** is not in mwBen:  -
**************^^^^^^^!!@@@@@*************** is not in mwBen:  09
**************^^^^^^^!!@@@@@*************** is not in mwBen:  2
**************^^^^^^^!!@@@@@*************** is not in mwBen:  29
**************^^^^^^^!!@@@@@*************** is not in mwBen:  3
**************^^^^^^^!!@@@@@*************** is not in mwBen:  47-3
**************^^^^^^^!!@@@@@*************** is not in mwBen:  53-4
**************^^^^^^^!!@@@@@*************** is not in mwBen:  DOCTER
**************^^^^^^^!!@@@@@*************** is not in mwBen:  PD1
**************^^^^^^^!!@@@@@*************** is not in mwBen:  PD2
derivative:  caricatured --- hw:  caricature
derivative:  characters --- hw:  character
derivative:  compared --- hw:  compare
**************^^^^^^^!!@@@@@*************** is not in mwBen:  dendrites
derivative:  dolls --- hw:  dol

In [ ]:
# Organize 'beautifulExcluHw.txt' and generate 'beautifulExclu.txt'
# (note: words in beautifulExcluHw.txt are absolutely hws in 39195 of mwl)
# directly from mwaled mongodb dpdict. words not being a hw in mwaled won't be added to the two lists

import pymongo
from pymongo import MongoClient

# codecs if for read and write files with unicode, encoding 'utf-8'
import codecs

#connect to MongoDB
client = MongoClient('localhost', 27017)
db = client.dpdb  # databas: dpdb
geweiDict = db.geweiDict201511 # collection (table): geweiDict201511
#geweiDict = db.geweiDictTest


with codecs.open("E:\\1Now\\mae\\beautifulExcluHw.txt", 'r', 'utf-8') as f:
    behws = f.read().splitlines()
    f.close()    
# sort and make unique words in beautifulExcluHw.txt
behws = sorted(set(behws))
print 'number of excluded hw: ', len(behws)
fd = codecs.open("E:\\1Now\\mae\\beautifulExcluHw.txt", 'w', 'utf-8')
for behw in behws:
    fd.write(behw + '\n')
fd.close()
    
fdict = codecs.open('E:\\1Now\\mae\\beautifulExcluraw.txt', 'w', 'utf-8')
    
results = geweiDict.find({}, {"_id":0, "hw":1, "infl":1, "subhw":1}) #.limit(1000)

# total number of head words
totalhw = 0
totalcnt = 0

for result in results:    
    word = result.get("hw")    

    if word and word in behws:
        #print word
#         fdict.write(word + ';' + word + '\n')
        fdict.write(word + '\n')
        totalhw += 1
    else:
#         print "no head word"
        continue

    # get items of result
    infls = result.get("infl")
    subhws = result.get("subhw")     
    
    if infls:
        for infl in infls:
#             fdict.write(infl + ';' + word + '\n')
            fdict.write(infl + '\n')
            totalcnt += 1   
            
    if subhws:
        for subhw in subhws:
            if subhw[0] != word[0]: continue # check if subhw is a real sub hw of word by comparing first letter
#             fdict.write(subhw + ';' + word  + '\n')
            fdict.write(subhw + '\n')
            totalcnt += 1        
    
fdict.close()
print "total head words: ", totalhw
print "total words: ", totalcnt + totalhw


# sort out by head words and make them unique
with codecs.open('E:\\1Now\\mae\\beautifulExcluraw.txt', 'r', 'utf-8') as f:
    lines = f.read().splitlines()
lines = sorted(set(lines))
#lines = sorted(lines)

fd = codecs.open('E:\\1Now\\mae\\beautifulExclu.txt', 'w', 'utf-8')
cnt = 0
for line in lines:
    fd.write(line + '\n')
    cnt += 1
fd.close()
print 'unique words:', cnt

In [12]:
import codecs

def compareWordlist(l1=[], l2=[], path='', f1='', f2=''):
    '''get common and different parts of two lists or word lists of two files'''
    
    if l1:
        words1 = l1
#         words1 = [x.strip() for x in l1]
        f1name = 'list-1'
    if l2:
        words2 = l2
#         words2 = [x.strip() for x in l2]
        f2name = 'list-2'
    if f1:
        with codecs.open(path+f1, 'r', 'utf-8') as f:
            words1 = f.read().splitlines()
#         words1 = [x.strip() for x in words1]
        f1name = f1
    if f2:
        with codecs.open(path+f2, 'r', 'utf-8') as f:
            words2 = f.read().splitlines()
#         words2 = [x.strip() for x in words2]
        f2name = f2
            
    comm = [x for x in words1 if x in words2]
    only1 = [x for x in words1 if x not in words2]
    only2 = [x for x in words2 if x not in words1]
    
    f = codecs.open('E:\\1Now\\taglist\\report\\compare_' + f1name + '_' + f2name + '.txt', 'w', 'utf-8')
    
    print '---common words of ' + f1name + ' and ' + f2name + '---: ' + str(len(comm))
    f.write('---common words of ' + f1name + ' and ' + f2name + '---: ' + str(len(comm)) + '\n')
    for x in comm:
        f.write(x + '\n')
    
    print '---words only in ' + f1name + '---: ' + str(len(only1))
    f.write('\n\n---words only in ' + f1name + '---: ' + str(len(only1)) + '\n')
    for x in only1:
        f.write(x + '\n')    
    
    print '---words only in ' + f2name + '---: ' + str(len(only2))
    f.write('\n\n---words only in ' + f2name + '---: ' + str(len(only2)) + '\n')
    for x in only2:
        f.write(x + '\n')
        
    f.close()

compareWordlist(path="E:\\1Now\\taglist\\", f1="84en_fr.txt", f2="mwlBenchmark.txt")

---common words of 84en_fr.txt and mwlBenchmark.txt---: 84
---words only in 84en_fr.txt---: 0
---words only in mwlBenchmark.txt---: 53957


In [2]:
# what are basic words of mwl?
import codecs
with codecs.open('E:\\1Now\\taglist\\3775BasicWordsfromDB.txt', 'r', 'utf-8') as f:
    lines = f.read().splitlines()
words = sorted(lines, key=lambda s: s.lower())
f = codecs.open('E:\\1Now\\taglist\\bwrawSort.txt', 'w', 'utf-8')
for word in words:
    f.write(word + '\n')
f.close()


In [47]:
import string
print 'walk'.capitalize() # Walk is good
print 'AWOL'.capitalize() # not good because Awol isn't a word
print '-ness'.capitalize()
print 'February'.capitalize()
al = string.lowercase[0:26]
print type(al)
# for x in al: print x
print 'Tuesday'[0] in al

Walk
Awol
-ness
February
<type 'str'>
False


In [8]:
import string
print string.uppercase[0:26]

ABCDEFGHIJKLMNOPQRSTUVWXYZ


In [10]:
# get duplicates
l = [1,2,3,4,4,5,5,6,1]
set([x for x in l if l.count(x) > 1])

{1, 4, 5}

In [74]:
# output core31.txt by processing core3000raw20151123.txt

with codecs.open('E:\\1Now\\taglist\\core3000raw20151123.txt', 'r', 'utf-8') as f:
    lines = f.read().splitlines()
words = [x.split('\t')[0] for x in lines]
wordsu = sorted(set(words))
fout = codecs.open('E:\\1Now\\taglist\\core3140n.txt', 'w', 'utf-8')
for word in wordsu:
    fout.write(word + '\n')
fout.close()

print len(words), len(wordsu)
dupl = set([x for x in words if words.count(x)>1])
print len(dupl)
# print sorted(dupl)

# # compare core3140n.txt and core31.txt
# with codecs.open('E:\\1Now\\taglist\\core31.txt', 'r', 'utf-8') as f:
#     core31s = f.read().splitlines()
#     f.close()
# print set(core31s) ^ set(wordsu)
    

3905 3140
661
set([])


In [1]:
def gethw():
    a = [{'k1': 'hello', 'k2':'hello2'}, {'k1': 'hi', 'k2':'hi2'}]
    return a
b = gethw()
for x in b:
    print x['k1'], '...', x['k2']

hello ... hello2
hi ... hi2


In [ ]:
print "thEy're café".title()
print "thEy're café".capitalize()


In [ ]:
a = "joy and café surprise.&nbsp;<br />And as I was ... surprise.And that"
print re.sub(r'[\<\>\/\&\.\;]+', ' ', a)
print u'é'

b = u"café laid-back, <code>that's</tag>" # u -- unicode 
c = "café laid-back, <code>that's</tag>"

xx = re.compile(r'[^\w\'\-]+', re.U)
print xx.sub('...', b) # work
print xx.sub('...', c) # doesn't work

print re.sub(r'[^A-Za-z\'\-]+', '...', b)
print re.sub(r'[^\w+]', '---', b, re.U) # why this one doesn't work?

In [2]:
import codecs
# is it necessary to close file in with statement?
with codecs.open('E:\\temp\\with.txt', 'w', 'utf-8') as f:
    f.write('hi')
    f.close() # no need, it'll be closed automatically
# f.write('hello') # f has been closed automatically
# f.close()

In [4]:
with codecs.open('E:\\1Now\\taglist\\84fr.txt', 'r', 'utf-8') as f:
    fr = f.read().splitlines()
with codecs.open('E:\\1Now\\taglist\\84fr2en.txt', 'r', 'utf-8') as f:
    fr2en = f.read().splitlines()
fd = codecs.open('E:\\1Now\\taglist\\84en_fr.txt', 'w', 'utf-8')
for i in range(0,84):
    fd.write(fr2en[i] + ';' + fr[i] + '\n')
fd.close()
    

In [17]:
with codecs.open('E:\\1Now\\taglist\\570awlraw.txt', 'r', 'utf-8') as f:
    lines = f.read().splitlines()
fd = codecs.open('E:\\1Now\\taglist\\570awl.txt', 'w', 'utf-8')
for x in lines:
    fd.write(x.split('\t')[0] + '\n')
fd.close()